In [1]:
import scipy.io as sio
from scipy.fft import fft
import os,glob
import math
import numpy as np
import pandas as pd
import os,glob
import pyexcel

In [2]:
from mat4py import loadmat, savemat

def get_neck_files(dirname, filepatern):
    files = []
    for filename in glob.iglob(os.path.join(dirname, filepatern), recursive=True):
        files.append(filename)
    return files

def wfft_hz(filename, Hz_Lthr, Hz_Hthr):
    '"Lthr: lower threshold; Hthr: high threshold"'
    data = loadmat(filename)
    fft_data = fft(data['data'])
    NFFT = math.floor(len(data['data'])/2)+1
    fft_data = abs(fft_data[1:NFFT])
    f = data['Fs']/2 * np.linspace(0, 1, NFFT)
    idx = [i for i, e in enumerate(f) if e <= Hz_Hthr] 
    X=f[idx]
    y=fft_data[idx]
    return X, y

In [3]:
folder_paths = ['../datasets/Age_A/Age_A_Male/', '../datasets/Age_A/Age_A_Female/']
age_group = []
sex = []
sub = []
side = []
filename = []
pats = ['**/*Neck_R_avg.mat', '**/*Neck_L_avg.mat']
Hz_Lthr, Hz_Hthr = 0.8, 2.5
for folder_path in folder_paths:
    for dirname in glob.iglob(os.path.join(folder_path, '**/')):
        for pat in pats:
            files = get_neck_files(dirname, pat)
            for file in files:
                [X,y] =  wfft_hz(file, Hz_Lthr, Hz_Hthr)
                data = {'Fs': 44100, 'data' : y.tolist()}
                savemat(file[0:-4]+'_'+str(Hz_Lthr)+'_'+str(Hz_Hthr)+'.mat', data)